<a href="https://colab.research.google.com/github/drdww/OPIM5641/blob/main/Module4/M4_2/0_Covering_Metropolis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integer Programming: Set Covering (Metropolis)
**OPIM 5641: Business Decision Modeling - University of Connecticut**

Related Reading:
*   Powell: Chapter 11 (Integer Optimization)

----------------------------------------



The set covering problem is a variation of the covering model in which the
variables are all binary. In addition, the parameters in the constraints are all zeroes
and ones.
In the classic version of the set covering problem, each project is described by a
subset of locations that it “covers.” The problem is to cover all locations with a minimal
number of projects.

We will try to minimize our 'activation variables' ($x$)- the number of police stations we are going to build in order to 'cover' the city.

Note that we are using an integer programming technique vs. a nonlinear/distance-based approach to this type of problem. If we had the centroid of each district, we could probably re-run this problem!

In [ ]:
# before you do anything...
# mount your drive!
# click folder on the left...
# import modules

%matplotlib inline
from pylab import *

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))

from pyomo.environ import *

# Problem Description
This example is from Powell - we will have a nice graphic for another set covering problem shortly.

The city of Metropolis is in the process of designing a new public emergency system, and their design
calls for locating emergency vehicles around the city. The city is divided into nine districts, and seven
potential sites have been identified as possible locations for emergency vehicles. Equipment located
at each potential site can reach some (but not all) of the districts within the 3-minute time
requirement specified by the city. In the following table, an entry of 1 means that the district can
be serviced from the corresponding site within the time requirement.  




Site |S1 | S2 | S3 | S4 | S5 | S6 | S7    
--- |---|--- | --- | --- |--- | --- |--- 
District D1 | 0 | 1 | 0 | 1 | 0 | 0 | 1 
District D2 | 1 | 0 | 0 | 0 | 0 | 1 | 1
District D3 | 0 | 1 | 0 | 0 | 0 | 1 | 1
District D4 | 0 | 1 | 1 | 0 | 1 | 1 | 0
District D5 | 1 | 0 | 1 | 0 | 1 | 0 | 0
District D6 | 1 | 0 | 0 | 1 | 0 | 0 | 1
District D7 | 1 | 0 | 0 | 0 | 0 | 0 | 1
District D8 | 0 | 0 | 1 | 1 | 1 | 0 | 0
District D9 | 1 | 0 | 0 | 0 | 1 | 0 | 0

The city wants to provide coverage to all nine districts within the specified time, using the minimum number of sites.


# Solution

* Here, each site can be associated with a set of districts that it covers.  
* For example, site S1 is associated with the set {0 1 0 0 1 1 1 0 1} that
appears as the first column of the table. If we select S1 to be a site for emergency equipment,
then we will have covered districts $D2, D5, D6, D7, and D9$.   
* For a solution to be feasible, we
would have to choose the remaining sites so that they cover $D1, D3, D4, and D8$.

**Define Objective Function**

Minimize $z = x_1 + x2 + x3 + x4  + x5 + x6 +  x7 $

**Constraints**

Subject to :
* $x_2 + x_4 + x_7  \geq 1$ `[District 1 is covered by sites 2, 4, 7]`
* $x_1 + x_6 + x_7  \geq 1$ `[District 2 is covered by sites 1, 6, 7]`
* $x_2 + x_6 + x_7  \geq 1$
* $x_2 + x_3 +  x_5 + x_6  \geq 1$
* $x_1 + x_3 + x_5  \geq 1$
* $x_1 + x_4 + x_6  \geq 1$
* $x_1  + x_7  \geq 1$
* $x_3 + x_4 + x_5  \geq 1$
* $x_1 + x_5  \geq 1$


`Domains`
* $x_i \in \{0,1\}, i \in [7]$  `(only binary activation variables)`




In [ ]:

# declare the model
model = ConcreteModel()

# decision variables (you lose some granularity in the constraints with this naming convention, but it is convenient!)
model.x = Var([1,2,3,4,5,6,7], domain=Binary)

x1 = model.x[1] # site 1
x2 = model.x[2] # site 2
x3 = model.x[3] 
x4 = model.x[4]
x5 = model.x[5] 
x6 = model.x[6]
x7 = model.x[7]


# objective
model.OBJ = Objective(expr = x1 + x2 + x3+ x4 + x5 + x6 + x7,
                      sense=minimize) 

# declare constraints
# we need to cover all of our neighborhoods
model.Constraint1 = Constraint(expr = x2 + x4 + x7 >= 1) # District 1 is covered by at least one of the following S2, S4, or S7
model.Constraint2 = Constraint(expr = x1 + x6 + x7 >= 1 )  # District 2 is covered by at least one of the following S1, S6, or S7
model.Constraint3 = Constraint(expr = x2 + x6 + x7 >= 1 )
model.Constraint4 = Constraint(expr = x2 + x3 + x5 + x6 >= 1 )
model.Constraint5 = Constraint(expr = x1 + x3 + x5 >= 1 )
model.Constraint6 = Constraint(expr = x1 + x4 + x6 >= 1 )
model.Constraint7 = Constraint(expr = x1 + x7 >= 1 )
model.Constraint8 = Constraint(expr = x3 + x4 + x5 >= 1 )
model.Constraint9 = Constraint(expr = x1 + x5 >=1 )


# show the model you've created
model.pprint()

1 Set Declarations
    x_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}

1 Var Declarations
    x : Size=7, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    OBJ : Size=1, Index=None, Active=True
        Key  : Active : Sense    : Expression
        None :   True : minimize : x[1] + x[2] + x[3] + x[4] + x[5] + x[6] + x[7]

9 Constraint Declarations
    Constraint1 : Size=1, Index=None, Active=True
        Ke

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("Number of stations = ", model.OBJ())
print ("X1 station = ", model.x[1]())
print ("X2 station = ", model.x[2]())
print ("X3 station = ", model.x[3]())
print ("X4 station = ", model.x[4]())
print ("X5 station = ", model.x[5]())
print ("X6 station = ", model.x[6]())
print ("X7 station = ", model.x[7]())


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3.0
  Upper bound: 3.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 7
  Number of binary variables: 7
  Number of integer variables: 7
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      

# Advanced Code Example
Here is a more concise way of solve the problem, but more Python-intensive.

In [ ]:
# declare the model
model = ConcreteModel()

stations = range(1,8)
Districts = [
              [2,4,7],
              [1,6,7],
              [2,6,7],
              [2,3,5,6],
              [1,3,5], 
              [1,4,6],
              [1,7],
              [3,4,5],
              [1,5]
]

# declare decision variables
model.x = Var(stations, domain=Binary)

obj_expr = 0
for station in stations:
  obj_expr += model.x[station]


# declare objective
model.n_stations = Objective(
                      expr = obj_expr, # values come from the table
                      sense = minimize)

# declare constraints
model.constraints = ConstraintList()

for x in range(0,9):
  expr = 0
  for y in Districts[x]:
    expr += model.x[y]
  model.constraints.add(expr >= 1)

# show the model you've created
model.pprint()

2 Set Declarations
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    9 : {1, 2, 3, 4, 5, 6, 7, 8, 9}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}

1 Var Declarations
    x : Size=7, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary

1 Objective Declarations
    n_stations : Size=1, Index=None, Active=True
        Key  : Active : Sense    : E

In [ ]:
# solve it
SolverFactory('cbc', executable='/usr/bin/cbc').solve(model).write()
# show the results
print("Number of stations = ", model.n_stations())
for station in stations:
  print("Station",station,":",model.x[station]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 3.0
  Upper bound: 3.0
  Number of objectives: 1
  Number of constraints: 9
  Number of variables: 7
  Number of binary variables: 7
  Number of integer variables: 7
  Number of nonzeros: 7
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      N